In [ ]:
from keras.layers import Input, Bidirectional, LSTM, Dense
from keras.models import Model
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.compat.v1.keras.optimizers import Adam


optimizer = Adam(learning_rate=0.05)




In [2]:
def replace_nans(dataset):
 

 indices = np.where(np.isnan(dataset))
 nan_indices = list(zip(indices))


 for i in range (len(indices[0])):
    dataset[indices[0][i]][indices[1][i]] = np.nanmean(dataset[:,indices[1][i]])
 return dataset    

In [3]:
#this function reads , stores and returns all data as numpy arrays
def read_all_data():
  #reading the training data
  df1 = pd.read_csv("C:\\Users\\91950\\OneDrive\\Desktop\\test_data_CNN_filter\\Zdata.csv")
  time_series_data = df1[['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18','f19','f20','f21','f22','f23','f24','f25','f26','f27','f28','f29','f30','f31','f32','f33','f34','f35','f36','f37','f38','f39','f40']].values
  time_series_data_1 = replace_nans(time_series_data)

  #reading  fdi data
  df2 = pd.read_csv("C:\\Users\\91950\\OneDrive\\Desktop\\test_data_CNN_filter\\Z_FDI.csv")
  fdi_test_data = df2[['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18','f19','f20','f21','f22','f23','f24','f25','f26','f27','f28','f29','f30','f31','f32','f33','f34','f35','f36','f37','f38','f39','f40']].values
  fdi_test_data_1= replace_nans(fdi_test_data)

  #reading trip_merge  data
  df3 = pd.read_csv("C:\\Users\\91950\\OneDrive\\Desktop\\test_data_CNN_filter\\Z_trip_merge.csv")
  trip_test_data = df3[['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18','f19','f20','f21','f22','f23','f24','f25','f26','f27','f28','f29','f30','f31','f32','f33','f34','f35','f36','f37','f38','f39','f40']].values
  trip_test_data_1 = replace_nans(trip_test_data)

  #reading fault data
  df4 = pd.read_csv("C:\\Users\\91950\\OneDrive\\Desktop\\test_data_CNN_filter\\Z_F_M2.csv")
  fault_test_data = df4[['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18','f19','f20','f21','f22','f23','f24','f25','f26','f27','f28','f29','f30','f31','f32','f33','f34','f35','f36','f37','f38','f39','f40']].values
  fault_test_data_1 = replace_nans(fault_test_data)

  return time_series_data_1, fdi_test_data_1, trip_test_data_1, fault_test_data_1

In [4]:
time_series_data, fdi_test_data, trip_test_data, fault_test_data = read_all_data()

In [5]:
#merging and shuffling steady state data and fault data 
merged_time_series_data = np.array((time_series_data.shape[0]+fault_test_data.shape[0],time_series_data.shape[1]))
merged_time_series_data = np.concatenate((time_series_data,fault_test_data))
np.random.shuffle(merged_time_series_data)

In [ ]:
 #normalizing all data
scalers = {}
for i in range(merged_time_series_data.shape[1]):
    scaler = MinMaxScaler(feature_range=(0, 1))
    merged_time_series_data[:, i] = scaler.fit_transform(merged_time_series_data[:, i].reshape(-1, 1)).flatten()
    scalers[f'feature{i + 1}'] = scaler

  #changing the data into 1 d array to be fed into input layer
new_time_series_data = merged_time_series_data.flatten()
new_time_series_data = np.reshape(new_time_series_data,(new_time_series_data.shape[0],1))
#splitting given data into training ,validation and testing data
train_data, temp_data = train_test_split(new_time_series_data, test_size=0.2, shuffle=False)
val_data, test_data = train_test_split(temp_data, test_size=0.5, shuffle=False)

#specifying dimensions of input
sequence_length = len(train_data)
# Creating the input layer
input_layer = Input(shape=(sequence_length, 1))
# Building  the encoder using Bidirectional LSTM
encoder_lstm1 = Bidirectional(LSTM(128, return_sequences=True))(input_layer)
encoder_lstm2 = Bidirectional(LSTM(64,return_sequences = True))(encoder_lstm1)
# Building the decoder using Bidirectional LSTM
decoder_lstm1 = Bidirectional(LSTM(64, return_sequences=True))(encoder_lstm2)
decoder_lstm2 = Bidirectional(LSTM(128, return_sequences=True))(decoder_lstm1)

# Adding a dense layer for mapping back to original dimension
input_dim = 1
decoded_output = Dense(input_dim, activation ='sigmoid')(decoder_lstm2)
# Creating the autoencoder model
autoencoder = Model(inputs=input_layer, outputs=decoded_output)
# Compiling the autoencoder
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
#summary of the autoencoder model
autoencoder.summary()

In [ ]:
#training the model
autoencoder.fit(train_data[0:100000], train_data[0:100000], epochs=14, batch_size=32, validation_split=0.2)
print("autoencoder model has been trained")

In [ ]:
#checking how the model performs on validation set to see if there's overfitting or underfitting
predicted_validation = autoencoder.predict(val_data[0:20000])
reconstruction_error_val = np.mean(np.square(predicted_validation - val_data[0:20000]), axis =1)


print(reconstruction_error_val)
print(np.mean(np.square(reconstruction_error_val)))

In [9]:
#defining a function that takes in test data and gives out reconstruction error of each data point in shape of original test data
def reconstruction_errors(test_dataset, num_features):

    #normalizing test dataset
    scalers = {}
    for i in range(test_dataset.shape[1]):
        scaler = MinMaxScaler(feature_range=(0, 1))
        test_dataset[:, i] = scaler.fit_transform(test_dataset[:, i].reshape(-1, 1)).flatten()
        scalers[f'feature{i + 1}'] = scaler
    
    #converting the normalized test dataset into 1d array to be fed into the autoencoder model
    flattened_test_dataset = test_dataset.flatten()
    flattened_test_dataset = np.reshape(flattened_test_dataset,(flattened_test_dataset.shape[0],1))

    #making predictions
    predicted_dataset = autoencoder.predict(flattened_test_dataset)
    #reshaping flattneed predicted dataset into original dimension as given in test dataset
    predicted_dataset = np.reshape(predicted_dataset,(int(predicted_dataset.shape[0] /num_features),int(num_features)))

    #defining reconstruction error matrix
    reconstruction_error = np.zeros_like(test_dataset, dtype=float)

    #calculating reconstruction error in each datapoint
    for k in range (test_dataset.shape[0]):
        for j in range(test_dataset.shape[1]):
            reconstruction_error[k][j] = np.mean(np.square(predicted_dataset[k][j] - test_dataset[k][j]))
    return reconstruction_error
    return np.mean(np.square(reconstruction_error))

In [10]:
#defining a function to calculate anomalies based on set  thresholds
def anomalies(reconstruction_error, threshold):
    num_anomalies = 0
    #reconstruction_error = reconstruction_errors("test_data",num_features)
    for i in range (reconstruction_error.shape[0]):
        for j in range (reconstruction_error.shape[1]):
            if(reconstruction_error[i][j] >threshold):
              num_anomalies = num_anomalies+1
              print(i+1,j+1)
    print(num_anomalies)

In [ ]:
num_features = 40
#calling functions to make predictions on test data (3 test dataset) and return reconstruction errors


recons_error_trip  = reconstruction_errors(trip_test_data,(num_features))
print("trip data reconstruction error ", np.mean(np.square(recons_error_trip)))
recons_error_fault = reconstruction_errors(fault_test_data[0:20000],(num_features))
print(" fault data reconstruction error ", np.mean(np.square(recons_error_fault)))

In [ ]:
recons_error_fdi   = reconstruction_errors(fdi_test_data,int(num_features))
print("fdi reconstruction error ", np.mean(np.square(recons_error_fdi)))

In [13]:
#output the reconstruction errors , helpful in debugging and understanding 
#with open('output.txt', 'w') as f:
#    f.write(np.array2string(recons_error_fdi))

In [ ]:
#setting threshold
threshold = 0.1e-4

In [ ]:
#calling functions to find and print indexes of detected anomalies
anomalies(recons_error_fault,threshold)

In [ ]:
anomalies(recons_error_fdi,threshold)

In [ ]:
anomalies(recons_error_trip,threshold)
